In [1]:
# Install required packages (run only once)
!pip install pyodbc pandas

import os
import shutil
import pandas as pd
import pyodbc
from datetime import datetime


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 1. SQL SERVER CONNECTION INFO
# -----------------------------
server = 'DESKTOP-V9VHBIC\\SQLEXPRESS'
database = 'ChatGpt_5_Projects'
driver = '{ODBC Driver 17 for SQL Server}'

conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
)
cursor = conn.cursor()

In [4]:
# 2. FOLDER PATHS
# -----------------------------
source_folder = r"C:\Users\Rajib\ChatGpt_5_Projects"
imported_folder = r"C:\Users\Rajib\ChatGpt_5_Projects\Imported"
cleaned_folder = r"C:\Users\Rajib\ChatGpt_5_Projects\CleanedFile"

# -----------------------------
# 3. FIND THE FIRST CSV IN SOURCE FOLDER
# -----------------------------
files = [f for f in os.listdir(source_folder) if f.endswith(".csv")]
if not files:
    raise FileNotFoundError("No CSV files found in the source folder!")

csv_file = os.path.join(source_folder, files[0])
print(f"Processing file: {csv_file}")

Processing file: C:\Users\Rajib\ChatGpt_5_Projects\sales_data.csv


In [5]:
# 4. BASIC CLEANING IN PYTHON
# -----------------------------
df = pd.read_csv(csv_file, parse_dates=["OrderDate"])
df

,OrderID,OrderDate,CustomerName,Category,Product,Quantity,UnitPrice
0,1001,2024-01-05,Alice Smith,Electronics,Headphones,2,50.0
1,1002,2024-01-06,Bob Johnson,Clothing,T-Shirt,3,20.0
2,1003,2024-02-02,Alice Smith,Electronics,Keyboard,1,80.0
3,1004,2024-02-10,Carol White,Furniture,Office Chair,1,150.0
4,1005,2024-02-15,David Brown,Electronics,Monitor,2,200.0
5,1006,2024-03-05,Eva Green,Clothing,Jeans,2,40.0
6,1007,2024-03-15,Frank Black,Furniture,Desk,1,300.0
7,1008,2024-04-01,Gina Lee,Electronics,Mouse,3,25.0
8,1009,2024-04-10,Henry Hall,Clothing,Jacket,1,120.0
9,1010,2024-04-12,Alice Smith,Furniture,Bookshelf,1,200.0


In [6]:
# Example: Drop duplicates, remove nulls
df.drop_duplicates(inplace=True)
df.dropna(subset=["OrderID", "OrderDate"], inplace=True)
df

,OrderID,OrderDate,CustomerName,Category,Product,Quantity,UnitPrice
0,1001,2024-01-05,Alice Smith,Electronics,Headphones,2,50.0
1,1002,2024-01-06,Bob Johnson,Clothing,T-Shirt,3,20.0
2,1003,2024-02-02,Alice Smith,Electronics,Keyboard,1,80.0
3,1004,2024-02-10,Carol White,Furniture,Office Chair,1,150.0
4,1005,2024-02-15,David Brown,Electronics,Monitor,2,200.0
5,1006,2024-03-05,Eva Green,Clothing,Jeans,2,40.0
6,1007,2024-03-15,Frank Black,Furniture,Desk,1,300.0
7,1008,2024-04-01,Gina Lee,Electronics,Mouse,3,25.0
8,1009,2024-04-10,Henry Hall,Clothing,Jacket,1,120.0
9,1010,2024-04-12,Alice Smith,Furniture,Bookshelf,1,200.0


In [7]:
# Example: Strip whitespace from string columns
for col in df.select_dtypes(include=['object']):
    df[col] = df[col].str.strip()

In [13]:
print(f"Rows after Python cleaning: {len(df)}")

Rows after Python cleaning: 10


In [17]:
df

,OrderID,OrderDate,CustomerName,Category,Product,Quantity,UnitPrice
0,1001,2024-01-05,Alice Smith,Electronics,Headphones,2,50.0
1,1002,2024-01-06,Bob Johnson,Clothing,T-Shirt,3,20.0
2,1003,2024-02-02,Alice Smith,Electronics,Keyboard,1,80.0
3,1004,2024-02-10,Carol White,Furniture,Office Chair,1,150.0
4,1005,2024-02-15,David Brown,Electronics,Monitor,2,200.0
5,1006,2024-03-05,Eva Green,Clothing,Jeans,2,40.0
6,1007,2024-03-15,Frank Black,Furniture,Desk,1,300.0
7,1008,2024-04-01,Gina Lee,Electronics,Mouse,3,25.0
8,1009,2024-04-10,Henry Hall,Clothing,Jacket,1,120.0
9,1010,2024-04-12,Alice Smith,Furniture,Bookshelf,1,200.0


In [18]:
# Remove extra spaces and standardize names
df.columns = df.columns.str.strip()        # remove leading/trailing spaces
df.columns = df.columns.str.replace(" ", "")  # remove internal spaces if needed
df.columns = df.columns.str.lower()        # make lowercase for consistency

print(df.columns.tolist())

['orderid', 'orderdate', 'customername', 'category', 'product', 'quantity', 'unitprice']


In [9]:
# -----------------------------
# 5. CREATE TABLE IF NOT EXISTS IN SQL SERVER
# -----------------------------
table_name = "SalesData"

cursor.execute(f"""
IF OBJECT_ID('{table_name}', 'U') IS NULL
BEGIN
    CREATE TABLE {table_name} (
        OrderID INT,
        OrderDate DATETIME,
        CustomerName NVARCHAR(255),
        Product NVARCHAR(255),
        Quantity INT,
        Price DECIMAL(10,2)
    )
END
""")
conn.commit()

In [19]:
# Importing the files

for index, row in df.iterrows():
    cursor.execute(f"""
    INSERT INTO {table_name} (OrderID, OrderDate, CustomerName, Product, Quantity, Price)
    VALUES (?, ?, ?, ?, ?, ?)
    """,
    row['orderid'],
    row['orderdate'],
    row['customername'],
    row['product'],
    row['quantity'],
    row['unitprice']  # lowercase now
    )

In [20]:
conn.commit()

In [21]:
cursor.execute(f"""
DELETE FROM {table_name}
WHERE Quantity <= 0 OR Price <= 0
""")
conn.commit()


In [22]:
pd.read_sql(f"SELECT * FROM {table_name}", conn)

C:\Users\Rajib\AppData\Local\Temp\ipykernel_23236\4111839161.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(f"SELECT * FROM {table_name}", conn)


,OrderID,OrderDate,CustomerName,Product,Quantity,Price
0,1001,2024-01-05,Alice Smith,Headphones,2,50.0
1,1002,2024-01-06,Bob Johnson,T-Shirt,3,20.0
2,1003,2024-02-02,Alice Smith,Keyboard,1,80.0
3,1004,2024-02-10,Carol White,Office Chair,1,150.0
4,1005,2024-02-15,David Brown,Monitor,2,200.0
5,1006,2024-03-05,Eva Green,Jeans,2,40.0
6,1007,2024-03-15,Frank Black,Desk,1,300.0
7,1008,2024-04-01,Gina Lee,Mouse,3,25.0
8,1009,2024-04-10,Henry Hall,Jacket,1,120.0
9,1010,2024-04-12,Alice Smith,Bookshelf,1,200.0


In [23]:
# 8. EXPORT CLEANED DATA TO CSV
# -----------------------------
df_sql = pd.read_sql(f"SELECT * FROM {table_name}", conn)
output_file = os.path.join(cleaned_folder, f"Cleaned_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
df_sql.to_csv(output_file, index=False)
print(f"Cleaned CSV saved to: {output_file}")

Cleaned CSV saved to: C:\Users\Rajib\ChatGpt_5_Projects\CleanedFile\Cleaned_20250809_200147.csv


C:\Users\Rajib\AppData\Local\Temp\ipykernel_23236\2925409435.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(f"SELECT * FROM {table_name}", conn)


In [24]:
# 9. MOVE ORIGINAL FILE TO IMPORTED FOLDER
# -----------------------------
shutil.move(csv_file, os.path.join(imported_folder, os.path.basename(csv_file)))
print(f"Original file moved to: {imported_folder}")

Original file moved to: C:\Users\Rajib\ChatGpt_5_Projects\Imported


In [25]:
# -----------------------------
# 10. CLOSE CONNECTION
# -----------------------------
cursor.close()
conn.close()

print("ETL process completed successfully!")

ETL process completed successfully!
